In [1]:
import json
import pandas as pd
import numpy as np
from mkdir import mk_dir
from bert_format import train_qa, test_qa, bset

# 构建训练集

In [ ]:
col_ll = [
    ["样本量","num",{"1":"How many patients are ivolved in this study?"}],
    ["患者性别","gender",{"1":"What's gender of the patient?"}],
    ["年龄","age",{"1":"How old is patient?"}],
    ["证据等级","evi",{"1":"Where is pathogen isolated?"}]]
file_dir_l = {"Bactology"}
for file_dir in file_dir_l:
    for col_l in col_ll:
        col_name = col_l[0]
        col_ab = col_l[1]
        que_d = col_l[2]
sped_path = "../5.Orgnize/文献标注-校对/"
sped_l = [ele for ele in os.listdir(sped_path) if col_ab in ele]
txt_path = "../4.Label_prepare/%s/" % file_dir
train_qa = []

for sped in sped_l:

    # 读取标注信息
    print(sped)
    df = pd.read_excel("%s/%s" % (sped_path, sped), header=0)
    df = df.loc[df["是否是致病菌"] == "是",:]

    for idx in df.index.tolist():

        # 创建WHAT Question样本
        if np.isnan(df.loc[idx,"old_start" ]) == False:
            if type(df.loc[idx,"answer" ]) == float:
                df.loc[idx,"answer" ] = "NaN"
                
            if df.loc[idx, "old_start" ] < len(df.loc[idx, "full"]):

                if df.loc[idx, "answer"][0] == df.loc[idx, "full"].replace("\n","")[int(df.loc[idx, "old_start" ])]:
                    k = int(df.loc[idx, "old_start" ])
                else:
                    k = int(df.loc[idx, "old_start" ]) + 1
            
                train_qa.append(train_qa(
                    str(df.loc[idx, "ID"]), 
                    que_d["1"],
                    df.loc[idx, "answer" ],k,
                    df.loc[idx, "full"].replace("\n","")))
                
bset(train_qa,"input/input_%s/" % (col_ab) ,"train_set.txt" )

# 构建预测集合

In [2]:
with open("../PubMed_Search/search_results/search_results_hanimals.txt", "r") as f:
    a = f.read()
    spec_papers_dic = json.loads(a)
# 加载判断了的病原菌信息
prediction = json.load(open("output/output_pat/total_predictions.json", 'r'))
result = {}
for i in prediction.keys():
    paper_id = i.split(":")[0]
    spec = i.split(":")[1]
    ncbi = "https://pubmed.ncbi.nlm.nih.gov/%s/" % paper_id
    result[i] = [paper_id, spec, prediction[i][1][0], ncbi]
result = pd.DataFrame(result, index = ["ID", "SPECIES", "rate", "link"]).T
prediction_extend = {}

for spec_type in spec_papers_dic:

    prediction_extend[spec_type] = {}
    
    for k, v in prediction.items():

        spec = k.split(":")[1]
        paper_id = k.split(":")[0]
        
        if spec in spec_papers_dic[spec_type]:

            if v[1][0] > 0.5:

                if spec not in prediction_extend[spec_type]:
                
                    prediction_extend[spec_type][spec] = {}
                    prediction_extend[spec_type][spec][paper_id] = spec_papers_dic[spec_type][spec][paper_id]
                    prediction_extend[spec_type][spec][paper_id]["predition"] = v
                else:
                    prediction_extend[spec_type][spec][paper_id] = spec_papers_dic[spec_type][spec][paper_id]
                    prediction_extend[spec_type][spec][paper_id]["predition"] = v

with open("output/output_pat/prediction_extend.json", "w") as f:
    json.dump(prediction_extend, f)

In [6]:
# 根据上述筛选出的样本的数据集
col_ll = [
    ["样本量","num",{"1":"How many patients are ivolved in this study?"}],
    ["患者性别","gender",{"1":"What's gender of the patient?"}],
    ["年龄","age",{"1":"How old is patient?"}],
    ["证据等级","evi",{"1":"Where is pathogen isolated?"}],
    ["免疫状态","immu",{"1":"What's patient immunity state?"}],
#     ["预后","outcome",{"1":"What's patient outcome?"}]
]
for col_l in col_ll:
    
    col_name = col_l[0]
    col_ab = col_l[1]
    que_d = col_l[2]
    
    test_qa = []

    for spec_type in spec_papers_dic:
        
        for pre_keys in prediction:
            
            if pre_keys.split(":")[1] in spec_papers_dic[spec_type]:

                if len(spec_papers_dic[spec_type][pre_keys.split(":")[1]][pre_keys.split(":")[0]]["texts"]["Ab"]) > 0:
                    test_qa.append(test_qa(
                        pre_keys,
                        que_d["1"].replace("pathogen", pre_keys),
                        spec_papers_dic[spec_type][pre_keys.split(":")[1]][pre_keys.split(":")[0]]["texts"]["Ab"]+" "+
                        " ".join(sum(spec_papers_dic[spec_type][pre_keys.split(":")[1]][pre_keys.split(":")[0]]["Meshhead"],[])) ))
    bset(test_qa,"input/input_%s/" % (col_ab),"test_set.json")